In [9]:
!pip install kafka-python
!pip install numpy

     |████████████████████████████████| 14.5 MB 1.7 MB/s eta 0:00:01    |████████▌                       | 3.9 MB 285 kB/s eta 0:00:38     |███████████████████████████████▎| 14.1 MB 1.7 MB/s eta 0:00:01


In [1]:
import os 

packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"
os.environ["PYSPARK_PYTHON"]="python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {0} pyspark-shell".format(packages))

In [2]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

KAFKA_BROKER = "kafka:9092"
KAFKA_TOPIC = "fixcer"

spark = SparkSession \
        .builder \
        .master("spark://spark-master:7077") \
        .appName("Kafka") \
        .getOrCreate()

In [ ]:
df = spark.read.format("json").option("mode", "FAILFAST") \
        .option("inferSchema", "true").load("hdfs://namenode/user/root/input/data10.json")

In [ ]:
df.printSchema()

In [ ]:
df = df.na.drop().dropDuplicates()

In [ ]:
df = df.filter(size(df['comments']) >= 30)

In [ ]:
import re

def convertString(string):
    try:
        string = re.sub('\D', '', string)
        return int(string)
    except:
        return 0
        

convert = udf(convertString, IntegerType())
df = df.withColumn("androidVersion", convert(col("androidVersion")))
df = df.withColumn("currentVersion", convert(col("currentVersion")))
df = df.withColumn("size", convert(col("size")))

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [39]:
df = df.withColumn('value' ,to_json(struct([df[x] for x in df.columns])))

df.selectExpr("CAST(title AS STRING) as key", "CAST(value AS STRING) as value").show(5)

+--------------------+--------------------+
|                 key|             valuess|
+--------------------+--------------------+
|KX Storm Team - N...|{"androidVersion"...|
|Quran Audio - Urd...|{"androidVersion"...|
|Reflexis ESS - Be...|{"androidVersion"...|
|          Műsorújság|{"androidVersion"...|
|            e-Título|{"androidVersion"...|
+--------------------+--------------------+
only showing top 5 rows



In [40]:
df.selectExpr("CAST(title AS STRING) as key", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BROKER) \
  .option("topic", KAFKA_TOPIC) \
  .save()